# Fish Muscle Stress Relaxation

Fit relaxation of fish muscle data with generalized and fractional Maxwell models.

**Data:** examples/data/pyRheo/fish_muscle/stressrelaxation_fishmuscle_data.csv


In [1]:
# Google Colab compatibility - uncomment if running in Colab
# !pip install -q rheojax
# from google.colab import drive
# drive.mount('/content/drive')


## Setup and Imports

In [2]:
# Configure matplotlib for inline plotting in VS Code/Jupyter
%matplotlib inline

import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from rheojax.core.data import RheoData
from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.fractional_maxwell_model import FractionalMaxwellModel
from rheojax.models.generalized_maxwell import GeneralizedMaxwell
from rheojax.models.herschel_bulkley import HerschelBulkley
from rheojax.pipeline.base import Pipeline
from rheojax.transforms.mastercurve import Mastercurve

jax, jnp = safe_import_jax()
verify_float64()
np.set_printoptions(precision=4, suppress=True)
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11
warnings.filterwarnings('ignore', category=RuntimeWarning)

def r2_complex(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    ss_res = np.sum(np.abs(y_true - y_pred) ** 2)
    ss_tot = np.sum(np.abs(y_true - np.mean(y_true)) ** 2)
    return float(1 - ss_res / ss_tot)

def mpe(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    return float(np.mean(np.abs(y_true - y_pred) / np.maximum(np.abs(y_true), 1e-12)) * 100)


INFO:2025-12-06 04:31:40,884:jax._src.xla_bridge:808: Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: dlopen(libtpu.so, 0x0001): tried: 'libtpu.so' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibtpu.so' (no such file), '/usr/lib/libtpu.so' (no such file, not in dyld cache), 'libtpu.so' (no such file)


Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: dlopen(libtpu.so, 0x0001): tried: 'libtpu.so' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibtpu.so' (no such file), '/usr/lib/libtpu.so' (no such file, not in dyld cache), 'libtpu.so' (no such file)


Loading rheojax version 0.4.0


/Users/b80985/Projects/rheojax/.venv/lib/python3.13/site-packages/piblin_jax/__init__.py:28: UserWarning: GPU acceleration is only available on Linux with CUDA 12+. Using JAX in CPU mode.
  from . import backend, data, dataio, transform


## Load relaxation data

In [3]:
DATA_DIR = Path.cwd().parent / 'data' / 'pyRheo' / 'fish_muscle'
df = pd.read_csv(DATA_DIR / 'stressrelaxation_fishmuscle_data.csv', sep='	')
df.columns = df.columns.str.strip()

# Columns: Time (s), Relaxation Modulus (Pa)
t = df.iloc[:,0].to_numpy()
sigma = df.iloc[:,1].to_numpy()

data = RheoData(x=t, y=sigma, x_units='s', y_units='Pa', domain='relaxation')
print(df.head())


     Time  Relaxation Modulus
0  0.0139             33600.0
1  0.0149             33243.0
2  0.0159             31100.0
3  0.0169             30900.0
4  0.0180             31700.0


## Fit relaxation models

In [4]:
gm = GeneralizedMaxwell(n_modes=5, modulus_type='tensile')
gm.fit(t, sigma, test_mode='relaxation', use_log_residuals=True, use_multi_start=True)
gm_pred = gm.predict(t)
gm_r2 = gm.score(t, sigma)

fm = FractionalMaxwellModel()
fm.fit(t, sigma, test_mode='relaxation', use_log_residuals=True)
fm_pred = fm.predict(t, test_mode='relaxation')
fm_r2 = r2_complex(sigma, fm_pred)

print({'gm_r2': gm_r2, 'fm_r2': fm_r2})


Starting least squares optimization | {'method': 'trf', 'n_params': 11, 'loss': 'linear', 'ftol': 1e-06, 'xtol': 1e-06, 'gtol': 1e-06}


Timer: optimization took 0.786693s


Convergence: reason=`ftol` termination condition is satisfied. | iterations=20 | final_cost=4.971501e+06 | time=0.787s | final_gradient_norm=35796038930.74231


Starting least squares optimization | {'method': 'trf', 'n_params': 11, 'loss': 'linear', 'ftol': 1e-06, 'xtol': 1e-06, 'gtol': 1e-06}


Timer: optimization took 0.224030s


...

Inner optimization loop hit iteration limit | {'inner_iterations': 100, 'actual_reduction': -1}


Timer: optimization took 1.313239s


Convergence: reason=Inner optimization loop exceeded maximum iterations. | iterations=1 | final_cost=6.142524e+03 | time=1.313s | final_gradient_norm=nan


NLSQ hit inner iteration limit; retrying with SciPy least_squares for stability.


{'gm_r2': 0.9996992915644162, 'fm_r2': -311.71653162797617}


## Plot relaxation fits

In [5]:
fig, ax = plt.subplots(figsize=(9, 6))
ax.loglog(t, sigma, 'o', label='Data', alpha=0.5)
ax.loglog(t, gm_pred, '-', label=f'Generalized Maxwell (R^2={gm_r2:.3f})')
ax.loglog(t, fm_pred, '--', label=f'Fractional Maxwell (R^2={fm_r2:.3f})')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Stress (Pa)')
ax.grid(True, which='both', ls='--', alpha=0.4)
ax.legend()
plt.show()


/var/folders/rq/lgpr8fvj349cr_wvkzv99md80000gp/T/ipykernel_15782/57055268.py:9: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
